In [83]:
#Use beautiful soup to retrive all MTA files from website (web scrape) just for 2020-01-01 to most recent

import re
import requests
import urllib.request
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd

url = 'http://web.mta.info/developers/turnstile.html'

#https://blog.proxypage.io/web-scrape-using-python-in-less-than-5-minutes-2/
#https://hackersandslackers.com/scraping-urls-with-beautifulsoup/
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

#print(soup.prettify()) you can see 'soup' is our entire html page

URL_PREFACE = 'http://web.mta.info/developers/'
a = soup.find_all('a')
o = ()
for i in range(37, len(a)):
    ins = (a[i].get_text(), URL_PREFACE + a[i]['href'],)
    o = o + (ins,)

col = ['C/A','UNIT','SCP','STATION','LINENAME','DIVISION','DATE','TIME','DESC','ENTRIES','EXITS']

for a, b in o: #for content, href in soup.find_all('a'), so for every tuple
    d = a[-4:] #notice the year is just the last 4 index
    print(d)
    if d.find('2020') !=-1:

        #what if I wanted to keep track of everything I'm downloading? How do I check
        #what's being pushed into df?
        content = requests.get(b).content
        data = pd.read_csv(b, names = col)
        print(data)
        df = DataFrame(data)
        df += df.append(data, ignore_index=True) #ignore index cause that will mess up col matrix size
        #data = pd.read_csv(filePath, sep=",", header=None, names = col_name)

df.count
df.head()

2021
2020
          C/A  UNIT       SCP        STATION  LINENAME  DIVISION        DATE  \
0         C/A  UNIT       SCP        STATION  LINENAME  DIVISION        DATE   
1        A002  R051  02-00-00          59 ST   NQR456W       BMT  12/19/2020   
2        A002  R051  02-00-00          59 ST   NQR456W       BMT  12/19/2020   
3        A002  R051  02-00-00          59 ST   NQR456W       BMT  12/19/2020   
4        A002  R051  02-00-00          59 ST   NQR456W       BMT  12/19/2020   
...       ...   ...       ...            ...       ...       ...         ...   
212295  TRAM2  R469  00-05-01  RIT-ROOSEVELT         R       RIT  12/25/2020   
212296  TRAM2  R469  00-05-01  RIT-ROOSEVELT         R       RIT  12/25/2020   
212297  TRAM2  R469  00-05-01  RIT-ROOSEVELT         R       RIT  12/25/2020   
212298  TRAM2  R469  00-05-01  RIT-ROOSEVELT         R       RIT  12/25/2020   
212299  TRAM2  R469  00-05-01  RIT-ROOSEVELT         R       RIT  12/25/2020   

            TIME     DESC    

/Users/user.me/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


NameError: name 'DataFrame' is not defined